# Ethereum Wallet Tutorial: Make your own wallet!
Monady 27, Aug, 2018, by Bowen

##  1. Environment setup 

 ***'ecdas'*** is an abbreviation of ***Elliptic Curve Digital Signature Algorithm***, which is used in ***Ethereum and Bitcoin***.

***'secp256k1'*** refers to the parameters of the elliptic curve used in ***Ethereum and Bitcoin***.

 ***'sha3'*** is an abbreviation of ***Secure Hash Algorithm 3***, which is used in ***Ehtereum(KECCAK-256 specifically)***.

 ***'web3'*** is an interface for interacting with the ***Ethereum*** blockchain and ecosystem.

***''JSON'***' (JavaScript Object Notation) is a lightweight data-interchange format.

In [1]:
#!/usr/bin/python3

# pip install ecdsa
# pip install pysha3
# pip install web3

from ecdsa import SigningKey, SECP256k1
import sha3
from web3 import Web3, HTTPProvider
import json
import time

## 2. Create your wallet locally (unconnected with Internet)

### 2.1 Generate a keccak256 curve

In [2]:
keccak = sha3.keccak_256()

### 2.2 Generate your private key and the corresponding public key randomly

In [3]:
private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key().to_string()

### 2.3 Update the keys to the curve

In [4]:
keccak.update(public_key)

### 2.4 Take the address string from the 24th bit

In [5]:
address = '0x' + keccak.hexdigest()[24:]

### 2.5 Show the keys and address

Attention: private keys is readable in hex format.

In [27]:
private_key.to_string()

b'\x194\xb6\x1az\x9d\xfb\xeb\xf8\xd3:\xab\x9b\xec\x96G\xca\xbf\xc7\xderSA\xef\x10G\x8e\x13\xdfNC\x93'

In [6]:
print('Your private key is:', '\n' + private_key.to_string().hex())
print()
print('Your public key is:', '\n' + public_key.hex())
print()
print('Your address is:', '\n' + address)
print()
print('Congratulations, you have an Ethereum wallet now!')

Your private key is: 
1934b61a7a9dfbebf8d33aab9bec9647cabfc7de725341ef10478e13df4e4393

Your public key is: 
b9be10aa189bb3921af7693b832638d6dbc67fbe267be6c411f7e30874dcd7a2ccf649c859b34e3ec24d47d22aafafa5f3ffb8eeecfc3064e658fcd8d3dff84a

Your address is: 
0x588d91c82a3bef5c2f4194b04588600568090837

Congratulations, you have an Ethereum wallet now!


## 3. Interact with Ethereum Blockchain

### Now you have a wallet, let's use it. 
### 3.1 Set up Infura and connect to the blockchain
***'Infura'*** is secure, reliable, and scalable (according to their website) access to Ethereum APIs and IPFS gateways.

In [7]:
passwd = "12345678"
eth_rpc = "https://mainnet.infura.io/FIOMnEEKo2wD2zF3JByq"

In [8]:
w3 = Web3(HTTPProvider(eth_rpc))

If it returns ***True***, it means that you are successfully connected with Ethereum.

In [11]:
w3.isConnected()

True

###  3.2 Send ETH

Check your balance

In [19]:
print('You have {} ETH.'.format(w3.eth.getBalance(w3.toChecksumAddress(address)) / 10**18))

You have 0.01 ETH.


Set up transaction parameters: send ***0.001*** ETH to ***0xf2c63217188a61aa10cff8a4a3d3399b4a8156b8***

ps: 'chainId : 1' means the main chain of Ethereum

In [21]:
amount_in_wei = int(0.001 * 10**18)
nonce = w3.eth.getTransactionCount(w3.toChecksumAddress(address))
txn_dict = {
        'to': w3.toChecksumAddress('0xf2c63217188a61aa10cff8a4a3d3399b4a8156b8'),
        'value': amount_in_wei,
        'gas': 21000,
        'gasPrice': w3.toWei('2', 'gwei'),
        'nonce': nonce,
        'chainId': 1
}

Sign the transaction with your ***private key***.

The can also be done ***without*** Internet, which is how ***cold*** wallet working.

In [28]:
signed_txn = w3.eth.account.signTransaction(txn_dict, private_key.to_string().hex())

txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

In [30]:
txn_hash

HexBytes('0x4da63425b41956c7c7ca7f246dc017ec20f027027e32395b6202bc3c87b473cf')

Now your balance will become ***Initila balance - sending amount - actual TX cost***, which is at least 

$$0.01 - 0.001 - \frac{21000\times2\times10^{9}}{10^{18}} = 0.01 - 0.001 - 0.000042 = 0.008958$$

In [33]:
0.01 - 0.001 - 21000*2*10**9/10**18

0.008958

In [34]:
w3.eth.getBalance(w3.toChecksumAddress(address)) / 10**18

0.008958

In [37]:
w3.eth.getTransaction('0x4da63425b41956c7c7ca7f246dc017ec20f027027e32395b6202bc3c87b473cf')

AttributeDict({'blockHash': HexBytes('0xb136d0b73996acf7fe1649000c17730320ab3efdf6b810ae5c83369a1812c5bd'),
 'blockNumber': 6218443,
 'from': '0x588D91C82A3BeF5c2f4194b04588600568090837',
 'gas': 21000,
 'gasPrice': 2000000000,
 'hash': HexBytes('0x4da63425b41956c7c7ca7f246dc017ec20f027027e32395b6202bc3c87b473cf'),
 'input': '0x',
 'nonce': 0,
 'r': HexBytes('0x589ece4149a8311ad06ad2b920708157e31154f783dba9eb289ed0f87bb011f4'),
 's': HexBytes('0x7b34e18983c59f5ba227dc384d7dad81ef4f1439e38afd24507df27351a3bc7d'),
 'to': '0xf2c63217188a61aA10CFf8a4a3d3399B4a8156B8',
 'transactionIndex': 33,
 'v': 37,
 'value': 1000000000000000})

###  3.2 Send tokens
Sending tokens is actually calling a contract. So we need to set up the contract parameters first.

ABI is short for ***Application Binary Interface***.

In [40]:
contract_address     = "0x3c136279fEde59bFE9a66Aa992e63b03baF1435E"
TOKEN_ABI = json.loads(
    """
    [{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"pausedPublic","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_value","type":"uint256"}],"name":"burn","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"pausedOwnerAdmin","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_subtractedValue","type":"uint256"}],"name":"decreaseApproval","outputs":[{"name":"success","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_value","type":"uint256"}],"name":"burnFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newAdmin","type":"address"}],"name":"changeAdmin","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_addedValue","type":"uint256"}],"name":"increaseApproval","outputs":[{"name":"success","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"token","type":"address"},{"name":"amount","type":"uint256"}],"name":"emergencyERC20Drain","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newPausedPublic","type":"bool"},{"name":"newPausedOwnerAdmin","type":"bool"}],"name":"pause","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"admin","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[{"name":"_admin","type":"address"},{"name":"_totalTokenAmount","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_burner","type":"address"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousAdmin","type":"address"},{"indexed":true,"name":"newAdmin","type":"address"}],"name":"AdminTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"newState","type":"bool"}],"name":"PausePublic","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"newState","type":"bool"}],"name":"PauseOwnerAdmin","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousOwner","type":"address"},{"indexed":true,"name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"owner","type":"address"},{"indexed":true,"name":"spender","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event"}]
    """
)
contract = w3.eth.contract(address=contract_address, abi=TOKEN_ABI)

Then it is similar to sending ETH.

In [51]:
contract.functions.balanceOf('0x588D91C82A3BeF5c2f4194b04588600568090837').call() / 10**18

10000000.0

Send 10,000 tokens to 0xf2c63217188a61aa10cff8a4a3d3399b4a8156b8.

In [61]:
nonce = w3.eth.getTransactionCount(w3.toChecksumAddress(address))
amount = 10000*10**18

# Build a transaction that invokes this contract's function, called transfer
token_txn = contract.functions.transfer(w3.toChecksumAddress('0xf2c63217188a61aa10cff8a4a3d3399b4a8156b8'), amount) \
            .buildTransaction({
            'chainId': 1,
            'gas': 42000,
            'gasPrice': w3.toWei('4', 'gwei'),
            'nonce': nonce,
            })

In [62]:
signed_token_txn = w3.eth.account.signTransaction(token_txn, private_key=private_key.to_string().hex())
token_txn_hash = w3.eth.sendRawTransaction(signed_token_txn.rawTransaction)

In [70]:
token_txn_hash

HexBytes('0x13857fe4df2f7131af41bcab902672e9743c1ac979ab452d8aaf35c30b637b67')

In [74]:
token_txn_receipt = w3.eth.getTransactionReceipt(token_txn_hash)

In [75]:
token_txn_receipt

AttributeDict({'blockHash': HexBytes('0xc2cb4ab9e81c27d7a4942206d8cf0a2fc846104547a1572ebdb61771736a24c1'),
 'blockNumber': 6218613,
 'contractAddress': None,
 'cumulativeGasUsed': 6116285,
 'from': '0x588d91c82a3bef5c2f4194b04588600568090837',
 'gasUsed': 38455,
 'logs': [AttributeDict({'address': '0x3c136279fEde59bFE9a66Aa992e63b03baF1435E',
   'blockHash': HexBytes('0xc2cb4ab9e81c27d7a4942206d8cf0a2fc846104547a1572ebdb61771736a24c1'),
   'blockNumber': 6218613,
   'data': '0x00000000000000000000000000000000000000000000021e19e0c9bab2400000',
   'logIndex': 81,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000588d91c82a3bef5c2f4194b04588600568090837'),
    HexBytes('0x000000000000000000000000f2c63217188a61aa10cff8a4a3d3399b4a8156b8')],
   'transactionHash': HexBytes('0x13857fe4df2f7131af41bcab902672e9743c1ac979ab452d8aaf35c30b637b67'),
   'transactionIndex': 112})],
 'logsBloom': 

Now your token balance will become ***Initila balance - sending amount***, which should be 

$$10000000 - 10000 = 9990000$$

In [76]:
contract.functions.balanceOf('0x588D91C82A3BeF5c2f4194b04588600568090837').call() / 10**18

9990000.0

Also your ETH balance becomes ***Initila balance  - actual TX cost***, which is 

$$0.008958 - \frac{38455\times4\times10^{9}}{10^{18}} = 0.008958 - 0.00015382 = 0.00880418$$

In [81]:
0.008958 - 38455*4*10**9/10**18

0.00880418

In [82]:
w3.eth.getBalance(w3.toChecksumAddress(address)) / 10**18

0.00880418